### QA EVCL (QA Evaluation)

In [43]:
import json
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
import pyro
import pyro.distributions as dist
from tqdm import tqdm
import os

# Load the JSON file
os.chdir('/home/pranav24/cs-546-project')
file_path = "/home/pranav24/cs-546-project/SSR/Latest_Weights/QA_Weights/task024_cosmosqa_answer_generation.json"
with open(file_path, "r") as f:
    data = json.load(f)

instruction="""\nInstruction: Craft one correct answer to the question given in input. Be straight to the point and do not give incorrect answers. Only output the answer without restating the question or context. Be consistent with the context and use common sense."""
final_instruction="Now, ensure to only give the answer and not restate the question or context. \nAnswer:"
# Extract input-output pairs from JSON
instances = data["Instances"][4500:5000]
test_inputs = [instruction+instance["input"]+final_instruction for instance in instances]
test_outputs = [instance["output"][0] for instance in instances]

# print(test_inputs[499])
# print()
# print(test_outputs[499])



# Tokenizer setup
base_model_path = "meta-llama/Meta-Llama-3-8B"  # Replace with actual model path
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Check if tokenizer has a padding token, if not, set the eos_token as padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    labels = tokenizer(
        examples["output"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs


# Create DataLoaders
batch_size = 16  # Adjust as needed


# Define the model and load weights
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
)
fine_tuned_weights_path = "/home/pranav24/cs-546-project/finetuned-weights-LoRA-EVCL-Correct-Task1_VCL_best"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, fine_tuned_weights_path)
pyro.get_param_store().load('pyro_param_store_task1_evcl_best.pt')

# Ensure compatibility with the unchanged part of the code
DEVICE = model.device

# Generate predictions
predictions = []
references = []
sampled_weights_log = []  # Store sampled weights

output_file_path = "/home/pranav24/cs-546-project/predictions_EVCL_Task1_FINAL_QA.json"

if os.path.exists(output_file_path):
    with open(output_file_path, "r") as f:
        saved_data = json.load(f)
else:
    saved_data = {"predictions": [], "references": []}

print("Generating predictions incrementally:")

print("Generating predictions:")

for i in tqdm(range(0, len(test_inputs), batch_size)):  # Loop in batches
    print(f'At sample {i}')
    batch_inputs = test_inputs[i:i + batch_size]
    batch_references = test_outputs[i:i + batch_size]

    # Tokenize the inputs in a batch
    inputs_tokenized = tokenizer(batch_inputs, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        with torch.cuda.amp.autocast():
            # Apply Pyro parameters to LoRA layers
            for name, module in model.named_modules():
                if hasattr(module, "lora_A"):
                    for key in module.lora_A:
                        loc = pyro.param(f"{name}.lora_A.{key}_loc")
                        scale = pyro.param(f"{name}.lora_A.{key}_scale")
                        sampled_weight = pyro.sample(
                            f"{name}.lora_A.{key}",
                            dist.Normal(loc, scale).to_event(loc.dim())
                        )
                        sampled_weights_log.append(
                            (name, key, sampled_weight.clone().cpu().numpy())
                        )
                        module.lora_A[key].weight.data.copy_(sampled_weight)

                if hasattr(module, "lora_B"):
                    for key in module.lora_B:
                        loc = pyro.param(f"{name}.lora_B.{key}_loc")
                        scale = pyro.param(f"{name}.lora_B.{key}_scale")
                        sampled_weight = pyro.sample(
                            f"{name}.lora_B.{key}",
                            dist.Normal(loc, scale).to_event(loc.dim())
                        )
                        sampled_weights_log.append(
                            (name, key, sampled_weight.clone().cpu().numpy())
                        )
                        module.lora_B[key].weight.data.copy_(sampled_weight)

            # Generate predictions using the tokenized inputs
            generated_ids = model.generate(
                input_ids=inputs_tokenized["input_ids"],
                attention_mask=inputs_tokenized["attention_mask"],
                max_new_tokens=30,  
                min_length=10,  
                no_repeat_ngram_size=2,  
                num_return_sequences=1,
                top_p=0.9,  
                temperature=0.7  
            )

        # Decode generated IDs
        batch_predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend(batch_predictions)
        references.extend(batch_references)

        saved_data["predictions"].extend(batch_predictions)
        saved_data["references"].extend(batch_references)

        # Save incrementally to JSON
        with open(output_file_path, "w") as json_file:
            json.dump(saved_data, json_file, indent=4)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Generating predictions incrementally:
Generating predictions:


  0%|          | 0/32 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_929/1212530307.py:106: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


At sample 0


  3%|▎         | 1/32 [00:06<03:20,  6.46s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

 94%|█████████▍| 30/32 [03:10<00:12,  6.28s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


At sample 480


 97%|█████████▋| 31/32 [03:16<00:06,  6.28s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


At sample 496


100%|██████████| 32/32 [03:21<00:00,  6.31s/it]


In [44]:
import evaluate

In [45]:
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)

# Display the results
print("\nROUGE Scores:")
print(results)


ROUGE Scores:
{'rouge1': np.float64(0.052607719254781), 'rouge2': np.float64(0.012274753261103251), 'rougeL': np.float64(0.041798541501681344), 'rougeLsum': np.float64(0.0469195144838407)}


In [39]:
# test_inputs=["Context: Then I drove all around to the used DVD stores to get season 2. ( which I found for 20 bucks! ) I have been watching a disk a night, and not going to bed until 3 am because I knew that the new season was starting soon and I didn't want to be behind. I just finished season 2 and was on the edge of my seat the whole last episode. Yesterday I taped what was on tv. Question: What does the narrator think about the price of the DVD set purchased?"]
import json
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
import pyro
import pyro.distributions as dist
from tqdm import tqdm

test_inputs=["\nInstruction: Craft one correct answer to the question given in input. Be straight to the point and do not give incorrect answers. Only output the answer without restating the question or context. Be consistent with the context and use common sense.\n Context: He would just flash me this amazing smile. the funny thing was i kept getting hit on. A couple people came up to me when i was standing there, a couple friends were flirting with me when i was smoking, and i was talking to this other guy that was dancing. I know i gave him my number, and i think it was right in front of him. Question: Why did the man smile widely at the speaker and proceed to hit on her? Now, ensure to only give the answer\nAnswer:"]
batch_size=1
final_answer=[]

base_model_path = "meta-llama/Meta-Llama-3-8B"

fine_tuned_weights_path="/home/pranav24/cs-546-project/finetuned-weights-LoRA-EVCL-Correct-Task1_VCL_best"
pyro.get_param_store().load('pyro_param_store_task1_evcl_best.pt')

tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Check if tokenizer has a padding token, if not, set the eos_token as padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    labels = tokenizer(
        examples["output"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs
# pyro.clear_param_store()

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, fine_tuned_weights_path)
# pyro.get_param_store().load('pyro_param_store_task1_vcl_best.pt')
# pyro.get_param_store().load('pyro_param_store_task1_vcl_best.pt')

# Ensure compatibility with the unchanged part of the code
DEVICE = model.device

sampled_weights_log=[]
for i in tqdm(range(0, len(test_inputs), batch_size)):  # Loop in batches
    batch_inputs = test_inputs[i:i + batch_size]
    # batch_references = test_outputs[i:i + batch_size]

    # Tokenize the inputs in a batch
    inputs_tokenized = tokenizer(batch_inputs, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        with torch.cuda.amp.autocast():
            # Apply Pyro parameters to LoRA layers
            for name, module in model.named_modules():
                if hasattr(module, "lora_A"):
                    for key in module.lora_A:
                        loc = pyro.param(f"{name}.lora_A.{key}_loc")
                        scale = pyro.param(f"{name}.lora_A.{key}_scale")
                        sampled_weight = pyro.sample(
                            f"{name}.lora_A.{key}",
                            dist.Normal(loc, scale).to_event(loc.dim())
                        )
                        sampled_weights_log.append(
                            (name, key, sampled_weight.clone().cpu().numpy())
                        )
                        module.lora_A[key].weight.data.copy_(sampled_weight)

                if hasattr(module, "lora_B"):
                    for key in module.lora_B:
                        loc = pyro.param(f"{name}.lora_B.{key}_loc")
                        scale = pyro.param(f"{name}.lora_B.{key}_scale")
                        sampled_weight = pyro.sample(
                            f"{name}.lora_B.{key}",
                            dist.Normal(loc, scale).to_event(loc.dim())
                        )
                        sampled_weights_log.append(
                            (name, key, sampled_weight.clone().cpu().numpy())
                        )
                        module.lora_B[key].weight.data.copy_(sampled_weight)

            # Generate predictions using the tokenized inputs
            # print(sampled_weights_log)
            generated_ids = model.generate(
                input_ids=inputs_tokenized["input_ids"],
                attention_mask=inputs_tokenized["attention_mask"],
                # max_length=512,  # Adjust as needed
                max_new_tokens=30,  # Restrict to a smaller maximum length
                min_length=10,  # Optional: Ensure a minimum length if needed
                no_repeat_ngram_size=2,  # Prevent repetitive n-grams
                num_return_sequences=1,
                top_p=0.9,  # Nucleus sampling: consider only top 90% probability mass
                temperature=0.7  # Control randomness (lower is more deterministic)
            )

        # Decode generated IDs
        final_batch_predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        final_answer.extend(final_batch_predictions)
        # predictions.extend(batch_predictions)
        # references.extend(batch_references)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_929/1617821873.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 1/1 [00:03<00:00,  3.99s/it]


In [40]:
final_answer

['\nInstruction: Craft one correct answer to the question given in input. Be straight to the point and do not give incorrect answers. Only output the answer without restating the question or context. Be consistent with the context and use common sense.\n Context: He would just flash me this amazing smile. the funny thing was i kept getting hit on. A couple people came up to me when i was standing there, a couple friends were flirting with me when i was smoking, and i was talking to this other guy that was dancing. I know i gave him my number, and i think it was right in front of him. Question: Why did the man smile widely at the speaker and proceed to hit on her? Now, ensure to only give the answer\nAnswer: The man was attracted to her because of her looks and personality. He wanted to get to know her better. \n']

In [28]:
# test_inputs=["Context: Then I drove all around to the used DVD stores to get season 2. ( which I found for 20 bucks! ) I have been watching a disk a night, and not going to bed until 3 am because I knew that the new season was starting soon and I didn't want to be behind. I just finished season 2 and was on the edge of my seat the whole last episode. Yesterday I taped what was on tv. Question: What does the narrator think about the price of the DVD set purchased?"]
import json
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from datasets import Dataset
from torch.utils.data import DataLoader
import torch
import pyro
import pyro.distributions as dist
from tqdm import tqdm

test_inputs=["Craft one correct answer to the question given in input. To make it more interesting, try to use non-stereotypical language if possible. Make sure your correct answer is reasonably long, consistent with the context, and requires common sense (instead of explicit extraction from the context.) In your answer, use as few words as possible from the given context. Use a response that is uncommon/non-stereotypical, so that it is less predictable. To be less repetitive, please vary your language for each question: Context: He would just flash me this amazing smile. the funny thing was i kept getting hit on. A couple people came up to me when i was standing there, a couple friends were flirting with me when i was smoking, and i was talking to this other guy that was dancing. I know i gave him my number, and i think it was right in front of him. Question: Why did the man smile widely at the speaker and proceed to hit on her?"]
batch_size=1
final_answer=[]

base_model_path = "meta-llama/Meta-Llama-3-8B"

fine_tuned_weights_path="/home/pranav24/cs-546-project/SSR/Latest_Weights/QA_Weights/finetuned-weights/QA_Final"
# pyro.get_param_store().load('pyro_param_store_task1_evcl_best.pt')

tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Check if tokenizer has a padding token, if not, set the eos_token as padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    labels = tokenizer(
        examples["output"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs
# pyro.clear_param_store()

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    quantization_config=bnb_config,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, fine_tuned_weights_path)
# pyro.get_param_store().load('pyro_param_store_task1_vcl_best.pt')
# pyro.get_param_store().load('pyro_param_store_task1_vcl_best.pt')

# Ensure compatibility with the unchanged part of the code
DEVICE = model.device

sampled_weights_log=[]
for i in tqdm(range(0, len(test_inputs), batch_size)):  # Loop in batches
    batch_inputs = test_inputs[i:i + batch_size]
    # batch_references = test_outputs[i:i + batch_size]

    # Tokenize the inputs in a batch
    inputs_tokenized = tokenizer(batch_inputs, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        with torch.cuda.amp.autocast():
            # Generate predictions using the tokenized inputs
            # print(sampled_weights_log)
            generated_ids = model.generate(
                input_ids=inputs_tokenized["input_ids"],
                attention_mask=inputs_tokenized["attention_mask"],
                # max_length=512,  # Adjust as needed
                max_new_tokens=30,  # Restrict to a smaller maximum length
                min_length=10,  # Optional: Ensure a minimum length if needed
                no_repeat_ngram_size=2,  # Prevent repetitive n-grams
                num_return_sequences=1,
                top_p=0.9,  # Nucleus sampling: consider only top 90% probability mass
                temperature=0.7  # Control randomness (lower is more deterministic)
            )

        # Decode generated IDs
        final_batch_predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        final_answer.extend(final_batch_predictions)
        # predictions.extend(batch_predictions)
        # references.extend(batch_references)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_929/2780875458.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


In [29]:
final_answer

['Craft one correct answer to the question given in input. To make it more interesting, try to use non-stereotypical language if possible. Make sure your correct answer is reasonably long, consistent with the context, and requires common sense (instead of explicit extraction from the context.) In your answer, use as few words as possible from the given context. Use a response that is uncommon/non-stereotypical, so that it is less predictable. To be less repetitive, please vary your language for each question: Context: He would just flash me this amazing smile. the funny thing was i kept getting hit on. A couple people came up to me when i was standing there, a couple friends were flirting with me when i was smoking, and i was talking to this other guy that was dancing. I know i gave him my number, and i think it was right in front of him. Question: Why did the man smile widely at the speaker and proceed to hit on her? Answer: The man was being flirtatious. \n']